# Answer checking/sentence processing from lexemes (stanza)

In [3]:
import pandas as pd
# from mht.scripts.LLM_scripts.post_processing import extract_single_sentence 

In [16]:
lippath = '/Users/pabloherrero/Documents/ManHatTan/mht/data/processed/LIPSTICK/hebrew_db_team.lip'
lip = pd.read_csv(lippath)
lip.set_index('word_ll', inplace=True, drop=False)
lip

,p_recall,n_id,timestamp,delta,user_id,learning_language,ui_language,word_ll,word_ul,lexeme_string,...,mdt_history,mdt_correct,mrt_history,mrt_correct,wdt_history,wdt_correct,wrt_history,wrt_correct,speed,rebag
word_ll,,,,,,,,,,,,,,,,,,,,,
מפלגה,0.333,201,1751016976,1043,pablo,iw,en,מפלגה,party,מפלגה/מפלגה<NOUN>Fem|Sing|,...,0,0,1,1,0,0,2,0,0.1426,False
המלצה,0.500,355,1751986534,970601,pablo,iw,en,המלצה,recommendation,"ה/ה<DET>Def|Art|,detמלצה/מלץ<NOUN>Fem|Sing|",...,1,0,0,0,0,0,1,1,0.0173,False
איום,0.750,123,1751017555,1622,pablo,iw,en,איום,threat,איום/איום<NOUN>Masc|Sing|,...,0,0,2,2,1,0,1,1,0.1271,False
שביתה,0.800,550,1753184844,2168911,pablo,iw,en,שביתה,strike,שביתה/שביתה<NOUN>Fem|Sing|,...,2,2,1,1,2,1,0,0,0.1107,False
צלחית,1.000,234,1751015991,58,pablo,iw,en,צלחית,saucer,צלחית/צלחית<ADV>,...,0,0,1,1,0,0,1,1,0.0735,False
מעריץ,1.000,501,1751015933,0,pablo,iw,en,מעריץ,admires,"מעריץ/העריץ<VERB>Masc|HIFIL|Sing|1,2,3|Part|Act|",...,2,2,1,1,0,0,0,0,0.1199,False


In [5]:
import stanza

# stanza.download('he', processors='tokenize,pos,lemma,depparse')
nlp = stanza.Pipeline('he', processors='tokenize,pos,lemma,depparse')

2025-07-29 14:12:38 WARNING: Language he package default expects mwt, which has been added
2025-07-29 14:12:38 INFO: Loading these models for language: he (Hebrew):
| Processor | Package |
-----------------------
| tokenize  | htb     |
| mwt       | htb     |
| pos       | htb     |
| lemma     | htb     |
| depparse  | htb     |

2025-07-29 14:12:38 INFO: Use device: cpu
2025-07-29 14:12:38 INFO: Loading: tokenize
/Users/pabloherrero/Library/Python/3.9/lib/python/site-packages/stanza/models/tokenization/trainer.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be exe

In [124]:
def get_lexeme(token, nlp=None) -> str:
    """    Returns a string representation of the lexeme for a given token.
    The format is: word/lemma<upos>feat1|feat2|...deprel, where:
    - word is the original word
    - lemma is the lemmatized form
    - upos is the universal part-of-speech tag
    - feats are the morphological features, separated by '|'
    - deprel is the dependency relation, if not 'root'
    """
    if not isinstance(token, str):
        raise ValueError("Input must be a string token.")
    if nlp is None:
        import stanza
        nlp = stanza.Pipeline('he', processors='tokenize,pos,lemma,depparse')
        
    doc = nlp(token)

    for sent in doc.sentences:
        lexeme_string = ''
        # print(len(sent.words))
        for word in sent.words:
            if word.lemma:
                lexeme_string += word.lemma 
            # if word.upos:
            #     lexeme_string += '<' + word.upos + '>'
            # if word.feats:
            #     feats = word.feats.split('|')
            #     lexeme_string += ''.join(feat.split('=')[1] +'|' for feat in feats)
            # if word.deprel != 'root':
            #     lexeme_string +=  ',' + word.deprel
    return lexeme_string


In [47]:
target_word = 'מפלגה'
generated_word = 'מפלגות'
lexeme_generated_word = get_lexeme(generated_word)
print(f"Lexeme for {generated_word}: {lexeme_generated_word}")
if lexeme_generated_word in lip.loc[target_word].lexeme_string:
    print('Correct lexeme')
lip.loc[target_word].lexeme_string

Lexeme for מפלגות: מפלגה
Correct lexeme


'מפלגה/מפלגה<NOUN>Fem|Sing|'

In [48]:
target_word = 'מפלגה'
generated_word = 'מסיבות'
lexeme_generated_word = get_lexeme(generated_word)
print(f"Lexeme for {generated_word}: {lexeme_generated_word}")
if lexeme_generated_word in lip.loc[target_word].lexeme_string:
    print('Correct lexeme')
else:
    print('Incorrect lexeme')
lip.loc[target_word].lexeme_string


Lexeme for מסיבות: מסיבה
Incorrect lexeme


'מפלגה/מפלגה<NOUN>Fem|Sing|'

In [49]:
target_word = 'מעריץ'
generated_word = 'מעריצים'
lexeme_generated_word = get_lexeme(generated_word)
print(f"Lexeme for {generated_word}: {lexeme_generated_word}")
if lexeme_generated_word in lip.loc[target_word].lexeme_string:
    print('Correct lexeme')
else:
    print('Incorrect lexeme')
lip.loc[target_word].lexeme_string

Lexeme for מעריצים: מעריץ
Correct lexeme


'מעריץ/העריץ<VERB>Masc|HIFIL|Sing|1,2,3|Part|Act|'

In [45]:
target_word = 'מעריץ'
generated_word = 'יעריצו'
lexeme_generated_word = get_lexeme(generated_word)
print(f"Lexeme for {generated_word}: {lexeme_generated_word}")
if lexeme_generated_word in lip.loc[target_word].lexeme_string:
    print('Correct lexeme')
else:
    print('Incorrect lexeme')

Lexeme for יעריצו: העריץ
Correct lexeme


# Check whole generated sentence

In [127]:
def check_lexeme_contains(sentence: str, word: str, lip: pd.DataFrame, nlp = None) -> tuple[bool, str]:
    """
    Checks if the translated sentence contains the lexeme of the specified word.
    Returns True if it does, False otherwise.
    """
    target_lip_lexeme = lip.loc[word].lexeme_string
    for word in sentence.split():
        lexeme_word = get_lexeme(word, nlp)
        if lexeme_word in target_lip_lexeme:
            sentence_blank = sentence.replace(word, '_' * len(word))
            return True, sentence_blank
    return False, None

In [134]:
sentence = 'אנשים מעריצים את הזמר'
word = 'מעריץ'
flag_contains, blank_sentence = check_lexeme_contains(sentence, word, lip, nlp=nlp)
if flag_contains:
    print(f"The sentence contains the lexeme for '{word}'.")
    print("Blanked sentence:", blank_sentence)
else:
    print(f"The sentence does not contain the lexeme for '{word}'.")

The sentence contains the lexeme for 'מעריץ'.
Blanked sentence: אנשים _______ את הזמר


# Check in other translations

In [102]:
from googletrans import Translator
async def translate_sentence(sentence: str, src_lang: str = 'en', dest_lang: str = 'he') -> str:
    """    Translates a sentence from the source language to the destination language.
    Uses Google Translate API asynchronously"""

    print(f'Starting translation, src = {sentence}, dest_lang = {dest_lang}')
    async with Translator() as translator:
        
        if not dest_lang:
            print('No destination language specified. Using Hebrew as default.')
            dest_lang = 'he'

        translation = await translator.translate(text = sentence, dest=dest_lang, src=src_lang)
        
        return translation


In [104]:
def extract_strings(nested):
    """   Extracts all strings from a nested structure (list or string).
    This function recursively traverses the structure and collects all strings.
    """
    strings = []
    if isinstance(nested, str):
        strings.append(nested)
    elif isinstance(nested, list):
        for item in nested:
            strings.extend(extract_strings(item))
    return strings

In [ ]:
# Example that yields two translation possibilities for "several"
extracted_sentence = 'I bought several helmets'
print("Extracted sentence:", extracted_sentence)
he_translation = await translate_sentence(extracted_sentence, src_lang='en', dest_lang='he')
print("Translation:", he_translation)

possible_translations_flat = extract_strings(he_translation.extra_data['possible-translations'][0])
print(possible_translations_flat)

Extracted sentence: I bought several helmets
Starting translation, src = I bought several helmets, dest_lang = he
Translation: Translated(src=en, dest=he, text=קניתי כמה קסדות, pronunciation=kaniti kma kasdot, extra_data="{'translat...")
['I bought several helmets', 'קניתי כמה קסדות', 'קניתי מספר קסדות', 'I bought several helmets']


In [ ]:
# Example that yields two different subjects (you singular and plural)
extracted_sentence = 'You bought several helmets'
print("Extracted sentence:", extracted_sentence)
he_translation = await translate_sentence(extracted_sentence, src_lang='en', dest_lang='he')
print("Translation:", he_translation)

possible_translations_flat = extract_strings(he_translation.extra_data['possible-translations'][0])
print(possible_translations_flat)

Extracted sentence: I bought several helmets
Starting translation, src = I bought several helmets, dest_lang = he
Translation: Translated(src=en, dest=he, text=קניתי כמה קסדות, pronunciation=kaniti kma kasdot, extra_data="{'translat...")
['I bought several helmets', 'קניתי כמה קסדות', 'קניתי מספר קסדות', 'I bought several helmets']


In [ ]:
# Translate sentence with two possible translations for "singer" (masculine and feminine)
extracted_sentence = 'People admire the singer'
print("Extracted sentence:", extracted_sentence)
he_translation = await translate_sentence(extracted_sentence, src_lang='en', dest_lang='he')
print("Translation:", he_translation)

possible_translations_flat = extract_strings(he_translation.extra_data['possible-translations'][0])
print(possible_translations_flat)

Extracted sentence: People admire the singer
Starting translation, src = People admire the singer, dest_lang = he
Translation finished
Translation: Translated(src=en, dest=he, text=אנשים מעריצים את הזמר, pronunciation=anshim ma'aritzim et hazmar, extra_data="{'translat...")
['People admire the singer', 'אנשים מעריצים את הזמר', 'אנשים מעריצים את הזמרת', 'People admire the singer']


In [105]:
# Translate sentence with several possible translations
extracted_sentence = 'What do you mean by "fair"?'
print("Extracted sentence:", extracted_sentence)
he_translation = await translate_sentence(extracted_sentence, src_lang='en', dest_lang='he')
print("Translation:", he_translation)

possible_translations_flat = extract_strings(he_translation.extra_data['possible-translations'][0])
print(possible_translations_flat)

Extracted sentence: What do you mean by "fair"?
Starting translation, src = What do you mean by "fair"?, dest_lang = he
Translation: Translated(src=en, dest=he, text=למה אתה מתכוון ב"הוגן "?, pronunciation=lema ata mitkavan be"hugan "?, extra_data="{'translat...")
['What do you mean by "fair"?', 'למה אתה מתכוון ב"הוגן "?', 'למה אתה מתכוון ב"היריד "?', 'What do you mean by "fair"?']


In [ ]:
def check_other_translations(he_translation, word_ll, lip, nlp=None):
    """
    Checks if the translation contains other possible translations for the word.
    Returns a list of possible translations if found, otherwise an empty list.
    """
    if not he_translation.extra_data or 'possible-translations' not in he_translation.extra_data:
        return False
    possible_tranlations = he_translation.extra_data['possible-translations'][0]
    possible_translations_flat = extract_strings(possible_tranlations)
    if not possible_translations_flat:
        return False
    
    # Iterate through the possible translations and check if any of them contain the lexeme of the word
    for possibility in possible_translations_flat:
        # For each sentence use check_lexeme_contains to iterate through the words
        if check_lexeme_contains(possibility, word_ll, lip, nlp):
            return True, possibility
    return False

In [ ]:
def test_check_other_translations():
    """
    Test the check_other_translations function with various inputs.
    """
    # Example translation with possible translations
    he_translation = {
        'extra_data': {
            'possible-translations': [
                ['אני קניתי כמה קסדות', 'אתה קנית כמה קסדות']
            ]
        }
    }
    
    word_ll = 'קסדה'
    lip = pd.DataFrame({
        'word_ll': ['קסדה'],
        'lexeme_string': ['קסדה/קסדה<NOUN>Number=Sing|Gender=Fem']
    }).set_index('word_ll')
    
    assert check_other_translations(he_translation, word_ll, lip) == True

    # Example translation with possible translations that do not match the lexeme
    he_translation = {
        'extra_data': {
            'possible-translations': [
                ['אני קניתי כמה מכנסיים', 'אתה קנית כמה מכנסיים']
            ]
        }
    }
    word_ll = 'קסדה'
    lip = pd.DataFrame({
        'word_ll': ['קסדה'],
        'lexeme_string': ['קסדה/קסדה<NOUN>Number=Sing|Gender=Fem']
    }).set_index('word_ll')

    assert check_other_translations(he_translation, word_ll, lip) == False

# Return blank instead of word:

In [118]:
def check_direct_contains(sentence, word: str) -> bool:
    """
    Checks if the translated sentence contains the specified word.
    Returns True if it does, False otherwise.
    """
    if word.lower() in sentence.text.lower():
        sentence_blank = sentence.text.replace(word, '_' * len(word)) #'_' * len(word))
        return True, sentence_blank
    else: 
        return False, None

In [ ]:
def test_check_direct_contains():
    """
    Test the check_direct_contains function with various inputs.
    """
    # Example sentence that contains the word
    class Sentence:
        def __init__(self, text):
            self.text = text
    sentence = Sentence("אני אוהב את התפוזים של הקיץ")
    word = "תפוזים"
    assert check_direct_contains(sentence, word) == (True, "אני אוהב את ה______ של הקיץ")


    # Example sentence that does not contain the word
    sentence = Sentence("אני אוהב את התפוחים")
    word = "תפוזים"
    assert check_direct_contains(sentence, word) == (False, None)

    # Example sentence where the word is present at the end of the sentence (weird ___ positioning)
    sentence = Sentence("אני אוהב את התפוזים")
    word = "תפוזים"
    print(check_direct_contains(sentence, word))
    assert check_direct_contains(sentence, word) == (True, "אני אוהב את ה______")

# Run the tests
test_check_direct_contains()

(True, 'אני אוהב את ה______')


# Extract random entry form lip_team

In [141]:
import random
lip.sample(1)

,p_recall,n_id,timestamp,delta,user_id,learning_language,ui_language,word_ll,word_ul,lexeme_string,...,mdt_history,mdt_correct,mrt_history,mrt_correct,wdt_history,wdt_correct,wrt_history,wrt_correct,speed,rebag
word_ll,,,,,,,,,,,,,,,,,,,,,
שביתה,0.8,550,1753184844,2168911,pablo,iw,en,שביתה,strike,שביתה/שביתה<NOUN>Fem|Sing|,...,2,2,1,1,2,1,0,0,0.1107,False
